In [1]:
import json

# colombia_geojson = json.load(open(colombia.geo.json"))
colombia_geojson = json.load(open("colombia.geo.small.json"))

for i, ft in enumerate(colombia_geojson["features"]):
    dpt = ft["properties"]["NOMBRE_DPT"]
    if dpt == "NARIÑO":
        print(i)

colombia_geojson["features"][16]["properties"]["NOMBRE_DPT"] = "NARINO"
colombia_geojson["features"][16]


16


{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-77.98359680174944, 2.577399968856929],
    [-78.04019927987088, 2.679899930951112],
    [-78.1092987064375, 2.59010005017951],
    [-78.18199920596432, 2.677599906788848],
    [-78.26439666730674, 2.658900022829962],
    [-78.28399658144323, 2.608700036505175],
    [-78.41819763171158, 2.525000095118564],
    [-78.44650268515674, 2.582299948272206],
    [-78.60669708265878, 2.481899976705108],
    [-78.67230224616438, 2.327899932410636],
    [-78.61979675289716, 2.152100086155233],
    [-78.71199798556334, 2.146100043735678],
    [-78.7182998655772, 2.02480006159856],
    [-78.66929626391334, 2.03620004643317],
    [-78.60870361277607, 1.919299960682365],
    [-78.61959838849143, 1.789299964641982],
    [-78.92340087822271, 1.827000022462836],
    [-79.07309722892839, 1.643900036388335],
    [-79.06040191565658, 1.58959996726236],
    [-78.69020080549383, 1.265699982929655],
    [-78.63819885214907, 1.19679999366

In [2]:
import pandas as pd

dpts = ['amazonas', 'antioquia']

precips = pd.DataFrame()

for dpt in dpts:
    precip = pd.read_csv(f"colombia_{dpt}_precip.csv", skiprows=3)
    precip['NOMBRE_DPT'] = dpt.upper()
    precip = precip.rename(columns={'Unnamed: 0': 'year'})
    precips = pd.concat([precips, precip])
precips

# precips.to_csv("precips_example.csv")

,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,NOMBRE_DPT
0,1901,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
1,1902,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
2,1903,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
3,1904,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
4,1905,232.02,215.38,286.78,323.91,311.61,273.50,246.93,218.34,204.54,243.73,218.11,224.09,AMAZONAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2016,136.53,129.38,194.03,198.58,396.10,306.57,152.96,372.74,282.54,319.36,287.16,190.67,ANTIOQUIA
116,2017,166.67,89.25,133.92,251.69,392.25,260.12,190.87,227.94,305.26,340.76,322.34,85.93,ANTIOQUIA
117,2018,86.30,95.46,181.34,211.68,392.52,216.75,193.21,173.51,290.55,293.48,285.10,167.14,ANTIOQUIA
118,2019,87.73,95.53,113.55,135.48,217.35,252.90,345.61,297.80,311.83,221.51,349.35,220.91,ANTIOQUIA


In [3]:
min_precip = precips.drop(columns=['year','NOMBRE_DPT']).dropna().to_numpy().min()
max_precip = precips.drop(columns=['year','NOMBRE_DPT']).dropna().to_numpy().max()
print(min_precip, max_precip)


23.43 709.09


In [4]:
jul_precips = precips[['Jul','NOMBRE_DPT']].groupby('NOMBRE_DPT').mean()
jul_precips.reset_index(inplace=True)
jul_precips

,NOMBRE_DPT,Jul
0,AMAZONAS,245.456218
1,ANTIOQUIA,225.933109


In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()
candidates = df.winner.unique()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.P("Candidate:"),
    dcc.RadioItems(
        id='candidate', 
        options=[{'value': x, 'label': x} 
                 for x in candidates],
        value=candidates[0],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="choropleth"),
])

@app.callback(
    Output("choropleth", "figure"), 
    [Input("candidate", "value")])
def display_choropleth(candidate):
    fig = px.choropleth(
        jul_precips, geojson=colombia_geojson, color="Jul",
        locations='NOMBRE_DPT', featureidkey="properties.NOMBRE_DPT",
        projection="mercator", range_color=[0, 6500])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

app.run_server(debug=True)



In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# fig = px.choropleth(
#     jul_precips, geojson=colombia_geojson, color="Jul",
#     locations='NOMBRE_DPT', featureidkey="properties.NOMBRE_DPT",
#    projection="mercator", range_color=[0, 6500])
# fig.update_geos(fitbounds="locations", visible=True)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(figure=fig)
])
app.run_server(debug=True)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)

fig = px.choropleth_mapbox(jul_precips, geojson=colombia_geojson, locations='NOMBRE_DPT', color='Jul',
                           color_continuous_scale="Viridis",
                           range_color=(min_precip, max_precip),
                           mapbox_style="carto-positron",
                           zoom=3,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": 74.2973328}
                          )


In [ ]:
import(go)

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="Colombia Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body',
        children=[


            dcc.Graph(
                id='map-plot3',
                figure={
                    'data': [go.Choroplethmapbox(
                        z = [1,2],
                        geojson=counties,
                        locations=locations,
                        colorscale='Viridis',
                        colorbar_title="Precipitation"
                    )],
                    'layout': go.Layout(
                        mapbox_style="carto-positron",
                        mapbox_accesstoken=token,
                        mapbox_zoom=4,
                        mapbox_center = {"lat": 4.570868, "lon": -74.2973328}
                    )
                }
            )
        ])])

app.run_server(mode='inline')

In [5]:
weather_dpts = []
for i, ft in enumerate(colombia_geojson["features"]):
    dpt = ft["properties"]["NOMBRE_DPT"]
    if dpt == "NARIÑO":
        print(i)
    weather_dpts.append(dpt)
    
weather_dpts.sort()
print(weather_dpts)

# cases = pd.read_csv("case.summary.noclimate.txt")
cases = pd.read_csv("cases.by.state.txt")
cases['admin_ascii'] = cases['admin_ascii'].str.upper()
cases_dpts = cases['admin_ascii'].unique()
cases_dpts.sort()
# print(cases_dpts)

# change case data dpt names to match geojson:
cases = cases.replace({"SAN ANDRES Y PROVIDENCIA": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
                       "BOGOTA": "SANTAFE DE BOGOTA D.C",
                       "VALLE": "VALLE DEL CAUCA",
                       "NARI�O": "NARINO"})
cases_dpts = cases['admin_ascii'].unique()
cases_dpts.sort()
print(cases_dpts)


['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ATLANTICO', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARINO', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SANTAFE DE BOGOTA D.C', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA']
['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOLIVAR' 'BOYACA' 'CALDAS'
 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO' 'CORDOBA' 'CUNDINAMARCA'
 'EXTERIOR' 'GUAINIA' 'GUAJIRA' 'GUAVIARE' 'HUILA' 'MAGDALENA' 'META'
 'NARINO' 'NORTE SANTANDER' 'PROCEDENCIA DESCONOCIDA' 'PUTUMAYO' 'QUINDIO'
 'RISARALDA' 'SAN ANDRES' 'SANTAFE DE BOGOTA D.C' 'SANTANDER' 'SUCRE'
 'TOLIMA' 'VALLE DEL CAUCA' 'VAUPES' 'VICHADA']


In [6]:
cases.head()
print(cases['year'].max())

some_cases = cases[(cases['month'] == 7.0) & (cases['year'] == 2017)]
some_cases.head()

min_cases = some_cases['total_cases'].min()
max_cases = some_cases['total_cases'].max()
print(min_cases, max_cases)


2017
2.0 476.0


In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

app = JupyterDash(__name__)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)

fig = px.choropleth_mapbox(some_cases, 
                           geojson=colombia_geojson, 
                           locations='admin_ascii', 
                           color='total_cases',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 800)

app.layout = html.Div([
    dcc.Graph(figure=fig)
])
app.run_server(debug=True)


try to make sliders:

In [7]:
some_cases = cases[cases['year'] == 2017]

min_cases = some_cases['total_cases'].min()
max_cases = some_cases['total_cases'].max()
print(min_cases, max_cases)

some_cases[some_cases['total_cases'] > 1000]

min_cases = max(0, min_cases - 100)
max_cases = max_cases + 100
print(min_cases, max_cases)


1.0 1267.0
0 1367.0


In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

app = JupyterDash(__name__)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)


fig = px.choropleth_mapbox(some_cases, 
                           geojson=colombia_geojson, 
                           locations='admin_ascii', 
                           color='total_cases',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 800)

app.layout = html.Div([
    dcc.Graph(figure=fig, id='graph-with-slider'),
    dcc.Slider(
        some_cases['month'].min(),
        some_cases['month'].max(),
        step=None,
        value=some_cases['month'].min(),
        marks={str(month): str(month) for month in some_cases['month'].unique()},
        updatemode="drag",
        id='month-slider')
])
app.run_server(debug=True)


@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('month-slider', 'value'))
def update_figure(selected_month):
    filt = some_cases[some_cases.month == selected_month]
    fig = px.choropleth_mapbox(filt, 
                               geojson=colombia_geojson, 
                               locations='admin_ascii', 
                               color='total_cases',
                               color_continuous_scale="Viridis",
                               range_color=(min_cases, max_cases),
                               mapbox_style="carto-positron",
                               zoom=4,
                               opacity=0.5,
                               featureidkey="properties.NOMBRE_DPT",
                               center = {"lat": 4.570868, "lon": -74.2973328},
                               height = 600)
    fig.update_layout(transition_duration=0)
    return fig


In [8]:
some_cases[some_cases['total_cases'].isnull()].head()


,month,year,admin_ascii,total_cases,confirmed_cases
354,1,2017,SANTAFE DE BOGOTA D.C,NaN,NaN
374,1,2017,PROCEDENCIA DESCONOCIDA,NaN,NaN
739,2,2017,SANTAFE DE BOGOTA D.C,NaN,NaN
759,2,2017,PROCEDENCIA DESCONOCIDA,NaN,NaN
768,2,2017,VAUPES,NaN,NaN


In [10]:
print(weather_dpts)

pop = pd.read_csv("Population.csv", sep=';')
pop['Department'] = pop['Department'].str.upper()
pop = pop.replace({"SAN_ANDRES_Y_PROVIDENCIA": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
                   "BOGOTA": "SANTAFE DE BOGOTA D.C",
                   "GUAJIRA": "LA GUAJIRA", 
                   "NORTE_DE_SANTANDER": "NORTE DE SANTANDER",
                   "VALLE_DEL_CAUCA": "VALLE DEL CAUCA"})
pop.sort_values(by=['Department'])
pop

['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ATLANTICO', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARINO', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SANTAFE DE BOGOTA D.C', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA']


,Department,Population,Rural,Urban
0,CUNDINAMARCA,"2,090,845",828215,2919060
1,ANTIOQUIA,"4,972,941","1,434,161","6,407,102"
2,VALLE DEL CAUCA,"3,809,542","666,344","4,475,886"
3,ATLANTICO,"2,404,831","130,686","2,535,517"
4,SANTANDER,"1,655,627","529,210","2,184,837"
5,BOLIVAR,"1,549,063","521,047","2,070,110"
6,CORDOBA,"937,319","847,464","1,784,783"
7,NARINO,"716,592","914,000","1,630,592"
8,NORTE DE SANTANDER,"1,173,712","317,977","1,491,689"
9,CAUCA,"545,902","918,586","1,464,488"


In [11]:
some_cases_all = cases[cases['year'].isin([2016, 2017])]
some_cases = cases[cases['year'] == 2016]
some_cases2 = cases[cases['year'] == 2017]

min_cases = some_cases_all['total_cases'].min()
max_cases = some_cases_all['total_cases'].max()
print(min_cases, max_cases)

min_cases = max(0, min_cases - 100)
max_cases = max_cases + 100
print(min_cases, max_cases)

some_cases_all[some_cases_all['total_cases'] > 1000]


1.0 5217.0
0 5317.0


,month,year,admin_ascii,total_cases,confirmed_cases
316,1,2016,ANTIOQUIA,2141.0,1362.0
346,1,2016,TOLIMA,1298.0,571.0
347,1,2016,VALLE DEL CAUCA,4025.0,1763.0
382,1,2017,VALLE DEL CAUCA,1267.0,291.0
701,2,2016,ANTIOQUIA,2044.0,1249.0
729,2,2016,SANTANDER,1087.0,395.0
732,2,2016,VALLE DEL CAUCA,5217.0,2154.0
1086,3,2016,ANTIOQUIA,1855.0,1116.0
1117,3,2016,VALLE DEL CAUCA,2838.0,1215.0
1471,4,2016,ANTIOQUIA,2520.0,1309.0


In [16]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

app = JupyterDash(__name__)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)

fig = px.choropleth_mapbox(some_cases, 
                           geojson=colombia_geojson, 
                           locations='admin_ascii', 
                           color='total_cases',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 800)
fig.update_layout(coloraxis_showscale = False)
fig2 = px.choropleth_mapbox(some_cases2, 
                           geojson=colombia_geojson, 
                           locations='admin_ascii', 
                           color='total_cases',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 800)

app.layout = html.Div([
    dcc.Slider(
        some_cases['month'].min(),
        some_cases['month'].max(),
        step=None,
        value=some_cases['month'].min(),
        marks={str(month): str(month) for month in some_cases['month'].unique()},
        updatemode="drag",
        id='month-slider'),
    html.Div(children=[
        dcc.Graph(figure=fig, id='graph-with-slider1', style={'display': 'inline-block'}),
        dcc.Graph(figure=fig2, id='graph-with-slider2', style={'display': 'inline-block'})])
])
app.run_server(debug=True)


@app.callback(
    Output('graph-with-slider1', 'figure'),
    Output('graph-with-slider2', 'figure'),
    Input('month-slider', 'value'))
def update_figure(selected_month):
    filt = some_cases[some_cases.month == selected_month]
    filt2 = some_cases2[some_cases2.month == selected_month]
    fig = px.choropleth_mapbox(filt, 
                               geojson=colombia_geojson, 
                               locations='admin_ascii', 
                               color='total_cases',
                               color_continuous_scale="Viridis",
                               range_color=(min_cases, max_cases),
                               mapbox_style="carto-positron",
                               zoom=4,
                               opacity=0.5,
                               featureidkey="properties.NOMBRE_DPT",
                               center = {"lat": 4.570868, "lon": -74.2973328},
                               height = 600)
    fig2 = px.choropleth_mapbox(filt2, 
                           geojson=colombia_geojson, 
                           locations='admin_ascii', 
                           color='total_cases',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 600)
    fig.update_layout(transition_duration=0, coloraxis_showscale = False)
    fig2.update_layout(transition_duration=0)
    return fig, fig2


/Users/kak4007/miniconda3/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/


In [31]:
def format_cases(cases_csv):
    cases = pd.read_csv(cases_csv)
    cases = cases.rename(columns={'Unnamed: 0': 'NOMBRE_DPT'})
    cases = pd.melt(cases, id_vars=['NOMBRE_DPT'], var_name='date', value_name='case_count')
    cases['NOMBRE_DPT'] = cases['NOMBRE_DPT'].str.upper()
    cases = cases.replace({"SAN ANDRES Y PROVIDENCIA": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
                           "GUAJIRA": "LA GUAJIRA"})
    cases['year'] = cases['date'].str.split("-").str[0].astype(int)
    cases['month'] = cases['date'].str.split("-").str[1].astype(int)
    min_year = cases.year.min()
    cases['month_id'] = cases.month + 12*(cases.year-min_year)
    cases['month_label'] = cases.month.astype(str) + "-" + cases.year.astype(str)
    return cases

# true = format_cases("True_ByRegion.csv")
# pred = format_cases("Predicted_ByRegion.csv")

true = format_cases("True_ByRegion_old.csv")
pred = format_cases("Predicted_ByRegion_old.csv")

display(true)
display(pred)


,NOMBRE_DPT,date,case_count,year,month,month_id,month_label
0,AMAZONAS,2015-01-01,53.0,2015,1,1,1-2015
1,ANTIOQUIA,2015-01-01,791.0,2015,1,1,1-2015
2,ARAUCA,2015-01-01,51.0,2015,1,1,1-2015
3,ATLANTICO,2015-01-01,351.0,2015,1,1,1-2015
4,BOLIVAR,2015-01-01,184.0,2015,1,1,1-2015
...,...,...,...,...,...,...,...
427,CAQUETA,2017-12-01,17.0,2017,12,36,12-2017
428,CAUCA,2017-12-01,9.0,2017,12,36,12-2017
429,CESAR,2017-12-01,37.0,2017,12,36,12-2017
430,CHOCO,2017-12-01,4.0,2017,12,36,12-2017


,NOMBRE_DPT,date,case_count,year,month,month_id,month_label
0,AMAZONAS,2015-01-01,23.452975,2015,1,1,1-2015
1,ANTIOQUIA,2015-01-01,1107.217365,2015,1,1,1-2015
2,ARAUCA,2015-01-01,179.648223,2015,1,1,1-2015
3,ATLANTICO,2015-01-01,1120.962352,2015,1,1,1-2015
4,BOLIVAR,2015-01-01,377.565236,2015,1,1,1-2015
...,...,...,...,...,...,...,...
427,CAQUETA,2017-12-01,115.877780,2017,12,36,12-2017
428,CAUCA,2017-12-01,14.680799,2017,12,36,12-2017
429,CESAR,2017-12-01,751.890186,2017,12,36,12-2017
430,CHOCO,2017-12-01,5.431656,2017,12,36,12-2017


In [32]:
print(weather_dpts)
print(pred.NOMBRE_DPT.unique())

['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ATLANTICO', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARINO', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SANTAFE DE BOGOTA D.C', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA']
['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOLIVAR' 'BOYACA' 'CALDAS'
 'CAQUETA' 'CAUCA' 'CESAR' 'CHOCO' 'CORDOBA']


In [34]:
cases = pd.concat([true, pred])

min_cases = cases.case_count.min()
max_cases = cases.case_count.max()
print(min_cases, max_cases)

min_cases = max(0, min_cases - 100)
max_cases = max_cases + 100
print(min_cases, max_cases)

cases[cases.case_count > 1000]


0.0 4127.122871939156
0 4227.122871939156


,NOMBRE_DPT,date,case_count,year,month,month_id,month_label
123,ATLANTICO,2015-11-01,1266.000000,2015,11,11,11-2015
145,ANTIOQUIA,2016-01-01,2141.000000,2016,1,13,1-2016
157,ANTIOQUIA,2016-02-01,2044.000000,2016,2,14,2-2016
169,ANTIOQUIA,2016-03-01,1855.000000,2016,3,15,3-2016
181,ANTIOQUIA,2016-04-01,2520.000000,2016,4,16,4-2016
193,ANTIOQUIA,2016-05-01,3904.000000,2016,5,17,5-2016
205,ANTIOQUIA,2016-06-01,3415.000000,2016,6,18,6-2016
217,ANTIOQUIA,2016-07-01,3062.000000,2016,7,19,7-2016
229,ANTIOQUIA,2016-08-01,3562.000000,2016,8,20,8-2016
241,ANTIOQUIA,2016-09-01,2011.000000,2016,9,21,9-2016


for confidence demo:

In [76]:
def format_cases(cases_csv):
    cases = pd.read_csv(cases_csv)
    cases = cases.rename(columns={'Unnamed: 0': 'NOMBRE_DPT'})
    cases = pd.melt(cases, id_vars=['NOMBRE_DPT'], var_name='date', value_name='case_count')
    cases['NOMBRE_DPT'] = cases['NOMBRE_DPT'].str.upper()
    # cases = cases.replace({"SAN ANDRES Y PROVIDENCIA": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
    #                        "GUAJIRA": "LA GUAJIRA"})
    cases['year'] = cases['date'].str.split("-").str[0].astype(int)
    cases['month'] = cases['date'].str.split("-").str[1].astype(int)
    min_year = cases.year.min()
    cases['month_id'] = cases.month + 12*(cases.year-min_year)
    cases['month_label'] = cases.month.astype(str) + "-" + cases.year.astype(str)
    return cases

true = format_cases("True_ByRegion.csv")
pred = format_cases("Predicted_ByRegion.csv")

# true = format_cases("True_ByRegion_old.csv")
# pred = format_cases("Predicted_ByRegion_old.csv")

display(true)
display(pred)






print(weather_dpts)
print(pred.NOMBRE_DPT.unique())





cases = pd.concat([true, pred])

min_cases = cases.case_count.min()
max_cases = cases.case_count.max()
print(min_cases, max_cases)

min_cases = max(0, min_cases - 100)
max_cases = max_cases + 100
print(min_cases, max_cases)

cases[cases.case_count > 1000]




from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

app = JupyterDash(__name__)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)

fig = px.choropleth_mapbox(true, 
                           geojson=colombia_geojson,
                           title="True",
                           locations='NOMBRE_DPT', 
                           color='case_count',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 800)
fig2 = px.choropleth_mapbox(pred, 
                           geojson=colombia_geojson,
                           title="Predicted",
                           locations='NOMBRE_DPT', 
                           color='case_count',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 800)
fig.update_layout(coloraxis_showscale = False, title_font_size=40, title_x=0.5)
fig2.update_layout(title_font_size=40, title_x=0.5)

app.layout = html.Div([
    dcc.Slider(
        true['month_id'].min(),
        true['month_id'].max(),
        step=None,
        value=true['month_id'].min(),
        marks={str(month): str(month) for month in true['month_id'].unique()},
        updatemode="drag",
        id='month-slider'),
    html.Div(children=[
        dcc.Graph(figure=fig, id='graph-with-slider1', style={'display': 'inline-block'}),
        dcc.Graph(figure=fig2, id='graph-with-slider2', style={'display': 'inline-block'})]),
    html.Div(id='date-label', style={'font-size':40, 'textAlign':'center'})
])
app.run_server(debug=True)


@app.callback(
    Output('graph-with-slider1', 'figure'),
    Output('graph-with-slider2', 'figure'),
    Input('month-slider', 'value'))
def update_figure(selected_month):
    filt = true[true.month_id == selected_month]
    filt2 = pred[pred.month_id == selected_month]
    fig = px.choropleth_mapbox(filt, 
                               geojson=colombia_geojson,
                               title="True",
                               locations='NOMBRE_DPT', 
                               color='case_count',
                               color_continuous_scale="Viridis",
                               range_color=(min_cases, max_cases),
                               mapbox_style="carto-positron",
                               zoom=4,
                               opacity=0.5,
                               featureidkey="properties.NOMBRE_DPT",
                               center = {"lat": 4.570868, "lon": -74.2973328},
                               height = 600)
    fig2 = px.choropleth_mapbox(filt2, 
                           geojson=colombia_geojson, 
                           title="Predicted",
                           locations="NOMBRE_DPT", 
                           color='case_count',
                           color_continuous_scale="Viridis",
                           range_color=(min_cases, max_cases),
                           mapbox_style="carto-positron",
                           zoom=4,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 600)
    fig.update_layout(transition_duration=0, coloraxis_showscale = False, title_font_size=40, title_x=0.5)
    fig2.update_layout(transition_duration=0, title_font_size=40, title_x=0.5)
    return fig, fig2

@app.callback(Output('date-label', 'children'), Input('month-slider', 'value'))
def update_label(selected_month):
    month_label = true[true.month_id == selected_month].iloc[0].month_label
    return str(month_label)


,NOMBRE_DPT,date,case_count,year,month,month_id,month_label
0,AMAZONAS,2015-01-01,53,2015,1,1,1-2015
1,ANTIOQUIA,2015-01-01,791,2015,1,1,1-2015
2,ARAUCA,2015-01-01,51,2015,1,1,1-2015
3,ATLANTICO,2015-01-01,351,2015,1,1,1-2015
4,BOLIVAR,2015-01-01,184,2015,1,1,1-2015
...,...,...,...,...,...,...,...
1111,SUCRE,2017-12-01,15,2017,12,36,12-2017
1112,TOLIMA,2017-12-01,153,2017,12,36,12-2017
1113,VALLE DEL CAUCA,2017-12-01,170,2017,12,36,12-2017
1114,VAUPES,2017-12-01,0,2017,12,36,12-2017


,NOMBRE_DPT,date,case_count,year,month,month_id,month_label
0,AMAZONAS,2015-01-01,23.452975,2015,1,1,1-2015
1,ANTIOQUIA,2015-01-01,1107.217365,2015,1,1,1-2015
2,ARAUCA,2015-01-01,179.648223,2015,1,1,1-2015
3,ATLANTICO,2015-01-01,1120.962352,2015,1,1,1-2015
4,BOLIVAR,2015-01-01,377.565236,2015,1,1,1-2015
...,...,...,...,...,...,...,...
1111,SUCRE,2017-12-01,335.898886,2017,12,36,12-2017
1112,TOLIMA,2017-12-01,912.961994,2017,12,36,12-2017
1113,VALLE DEL CAUCA,2017-12-01,1087.226544,2017,12,36,12-2017
1114,VAUPES,2017-12-01,6.944178,2017,12,36,12-2017


['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ATLANTICO', 'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAINIA', 'GUAVIARE', 'HUILA', 'LA GUAJIRA', 'MAGDALENA', 'META', 'NARINO', 'NORTE DE SANTANDER', 'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SANTAFE DE BOGOTA D.C', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE DEL CAUCA', 'VAUPES', 'VICHADA']
['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'BOLIVAR' 'BOYACA' 'CALDAS'
 'CAQUETA' 'CAUCA' 'CESAR' 'CHOCO' 'CORDOBA' 'CUNDINAMARCA' 'GUAINIA'
 'GUAJIRA' 'GUAVIARE' 'HUILA' 'MAGDALENA' 'META' 'NARINO'
 'NORTE DE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA'
 'SAN ANDRES Y PROVIDENCIA' 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA'
 'VAUPES' 'VICHADA']
0.0 5217.0
0 5317.0


/Users/kak4007/miniconda3/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/


In [78]:
pred2 = pred.copy()
pred2['prev_month_id'] = pred2.month_id - 1
# pred.loc[pred['prev_month_id']]

pred2 = pd.merge(pred2, 
                true[['month_id','NOMBRE_DPT','case_count']], 
                left_on=['prev_month_id','NOMBRE_DPT'], 
                right_on=['month_id','NOMBRE_DPT'])
pred2['pred_delta'] = pred2.case_count_x - pred2.case_count_y
display(pred2.head())


max_delta = pred2.pred_delta.max() + 10
min_delta = pred2.pred_delta.min() - 10
min_delta = max(min_delta, max_delta*-1)
print(min_delta, max_delta)


from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

app = JupyterDash(__name__)

token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
px.set_mapbox_access_token(token)

fig = px.choropleth_mapbox(pred2, 
                           geojson=colombia_geojson,
                           title="Predicted Case Changes (Next Month)",
                           custom_data=['pred_delta'],
                           locations='NOMBRE_DPT', 
                           color='pred_delta',
                           color_continuous_scale="Thermal",
                           range_color=(min_delta, max_delta),
                           mapbox_style="carto-positron",
                           zoom=4.5,
                           opacity=0.5,
                           featureidkey="properties.NOMBRE_DPT",
                           center = {"lat": 4.570868, "lon": -74.2973328},
                           height = 600)
fig.update_layout(title_font_size=40, title_x=0.5, coloraxis_colorbar=dict(title=""))
fig.update_traces(hovertemplate="<br>".join([
    "Region: %(location)",
    "Predicted change: %(customdata[0])"]))

app.layout = html.Div([
    dcc.Slider(
        pred2['month_id_x'].min(),
        pred2['month_id_x'].max(),
        step=None,
        value=pred2['month_id_x'].min(),
        marks={str(month): str(month) for month in pred2['month_id_x'].unique()},
        updatemode="drag",
        id='month-slider'),
    dcc.Graph(figure=fig, id='graph-with-slider1'),
    html.Div(id='date-label', style={'font-size':40, 'textAlign':'center'})
])
app.run_server(debug=True)


@app.callback(
    Output('graph-with-slider1', 'figure'),
    Input('month-slider', 'value'))
def update_figure(selected_month):
    filt = pred2[pred2.month_id_x == selected_month]
    fig = px.choropleth_mapbox(filt, 
                               geojson=colombia_geojson,
                               title="Predicted Case Changes (Next Month)",
                               custom_data=['pred_delta'],
                               locations='NOMBRE_DPT', 
                               color='pred_delta',
                               color_continuous_scale="Thermal",
                               range_color=(min_delta, max_delta),
                               mapbox_style="carto-positron",
                               zoom=4,
                               opacity=0.5,
                               featureidkey="properties.NOMBRE_DPT",
                               center = {"lat": 4.570868, "lon": -74.2973328},
                               height = 600)
    fig.update_layout(transition_duration=0, title_font_size=40, title_x=0.5, coloraxis_colorbar=dict(title=""))
    fig.update_traces(hovertemplate="<br>".join([
        "Region: %{location}",
        "Predicted change: %{customdata[0]}"]))
    return fig

@app.callback(Output('date-label', 'children'), Input('month-slider', 'value'))
def update_label(selected_month):
    month_label = pred2[pred2.month_id_x == selected_month].iloc[0].month_label
    return str(month_label)


,NOMBRE_DPT,date,case_count_x,year,month,month_id_x,month_label,prev_month_id,month_id_y,case_count_y,pred_delta
0,AMAZONAS,2015-02-01,13.739869,2015,2,2,2-2015,1,1,53,-39.260131
1,ANTIOQUIA,2015-02-01,807.101795,2015,2,2,2-2015,1,1,791,16.101795
2,ARAUCA,2015-02-01,163.580805,2015,2,2,2-2015,1,1,51,112.580805
3,ATLANTICO,2015-02-01,1232.968595,2015,2,2,2-2015,1,1,351,881.968595
4,BOLIVAR,2015-02-01,473.176055,2015,2,2,2-2015,1,1,184,289.176055


-2297.2545776118704 2297.2545776118704
Dash app running on http://127.0.0.1:8050/


/Users/kak4007/miniconda3/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

